## Load the data set


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

2023-12-12 02:12:05.139103: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12 02:12:05.184645: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 02:12:05.184677: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 02:12:05.184700: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 02:12:05.193557: I tensorflow/core/platform/cpu_feature_g

In [ ]:
train, test =  tfds.load("oxford_iiit_pet",split=["train","test"],as_supervised=True)

2023-12-12 02:13:16.965029: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14774 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-12-12 02:13:16.965656: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10668 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:73:00.0, compute capability: 7.5


## Model

In [ ]:
base_model=tf.keras.applications.ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3),pooling="avg")

In [4]:
inputs=tf.keras.layers.Input(shape=(None,None,3),name="input")
resizing=tf.keras.layers.Resizing(224,224)(inputs)
preprocessed=tf.keras.applications.resnet.preprocess_input(resizing)
embeddings=base_model(preprocessed)
output=tf.keras.layers.Flatten()(embeddings)
model=tf.keras.Model(inputs=inputs,outputs=output)


In [5]:
train_ds=train.map(lambda x,y:(tf.expand_dims(x,axis=0),y))
# train_ds=train.batch(1)
train_ds=train_ds.map(lambda x,y:(model(x),y))

test_ds=test.map(lambda x,y:(tf.expand_dims(x,axis=0),y))
# test_ds=test.batch(1) 
test_ds=test_ds.map(lambda x,y:(model(x),y))


In [6]:
train_x=[]
train_y=[]
for x,y in train_ds:
    train_x.append(x[0])
    train_y.append(y)

Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment


In [7]:
test_x=[]
test_y=[]
for x,y in test_ds:
    test_x.append(x[0])
    test_y.append(y)
    

In [8]:
from sklearn.neighbors import KNeighborsClassifier

classifer=KNeighborsClassifier(n_neighbors=10,weights="distance",algorithm="auto")
classifer.fit(train_x,train_y)
classifer.score(test_x,test_y)

0.8383755791768874

In [10]:
weights=["distance","uniform"]  
max_score=0
best_n=0

for n in range(1,101):
    classifer=KNeighborsClassifier(n_neighbors=n,weights="distance",algorithm="auto")
    classifer.fit(train_x,train_y)
    score=classifer.score(test_x,test_y)
    if score>max_score:
        max_score=score
        best_n=n
print(f"Best accuracy is {max_score} with  {best_n} neighbors")

Best accuracy is 0.84518942491142 with  47 neighbors


In [11]:
max_score=0
best_n=0
for n in range(1,101):
    classifer=KNeighborsClassifier(n_neighbors=n,weights="uniform",algorithm="auto")
    classifer.fit(train_x,train_y)
    score=classifer.score(test_x,test_y)
    if score>max_score:
        max_score=score
        best_n=n
print(f"Best accuracy is {max_score} with  {best_n} neighbors")

Best accuracy is 0.842463886617607 with  25 neighbors
